In [1]:
import numpy as np
'''
Simulates cartpole starting at x0 with action u -- ported from CS287 Matlab code
'''

def sim_cartpole(x0, u, dt):
    
    def dynamics(x, u):
        mc = 10
        mp = 1
        l = 0.5
        g = 9.81
        T = 0.25
        s = np.sin(x[1])
        c = np.cos(x[1])
        
        xddot = (u + np.multiply(mp*s, l*np.power(x[3],2) + g*c))/(mc + mp*np.power(s,2))
        tddot = (-u*c - np.multiply(np.multiply(mp*l*np.power(x[3],2), c),s) - 
                 np.multiply((mc+mp)*g,s)) / (l * (mc + np.multiply(mp, np.power(s,2))))
        xdot = x[2:4]
        xdot = np.append(xdot, xddot)
        xdot = np.append(xdot, tddot)
        
        return xdot
    
    DT = 0.1
    t = 0
    while t < dt:
        current_dt = min(DT, dt-t)
        x0 = x0 + current_dt * dynamics(x0, u)
        t = t + current_dt
    
    return x0
    
 

In [2]:
'''
Linearizes the dynamics of cartpole around a reference point for use in an LQR controler
'''

def linearize_cartpole(x_ref, u_ref, dt, eps):
    A = np.zeros([4,4])

    for i in range(4):
        increment = np.zeros([4,])
        increment[i] = eps
        A[:,i] = (sim_cartpole(x_ref + increment, u_ref, dt) - 
                  sim_cartpole(x_ref, u_ref, dt)) / (eps)
    
    B = (sim_cartpole(x_ref, u_ref + eps, dt) - sim_cartpole(x_ref, u_ref, dt)) / (eps)
    
    c = x_ref
    
    return A, B, c

In [3]:
'''
Computes the LQR infinte horizon controller associated with linear dyamics A, B and quadratic cost Q, R

NOTE: Current version only works for cartpole because I hardcoded a couple of numbers for now
'''

def lqr_infinite_horizon(A, B, Q, R):
    nA = A.shape[0]

    if len(B.shape) == 1:
        nB = 1
    else:
        nB = B.shape[1]

    P_current = np.zeros([nA, nA])

    P_new = np.eye(nA)

    K_current = np.zeros([nB, nA])

    K_new= np.triu(np.tril(np.ones([nB,nA]),0),0)

    while np.linalg.norm(K_new - K_current, 2) > 1E-4:
        P_current = P_new
      
        K_current = K_new
        
        K_new = -np.linalg.inv(R + np.dot(np.dot( np.transpose(B), 
                                                  P_current), 
                                                  B)) * np.dot(np.dot( np.transpose(B), 
                                                                       P_current), 
                                                                       A)

        P_new = Q + np.dot(np.dot( np.transpose(K_new), 
                                   R), 
                                   K_new) + np.dot(np.dot( np.transpose(A + np.dot(B.reshape(4,1), K_new)),
                                                           P_current),
                                                           (A + np.dot(B.reshape(4,1), K_new.reshape(1,4)))
                          )
        
    return K_new, P_new

In [5]:
x_ref = np.array([0, np.pi, 0, 0])
A, B, c = linearize_cartpole(x_ref, 0, 0.1, 0.1)
Q = np.eye(4)
R = np.eye(1)

K_inf, P_inf = lqr_infinite_horizon(A, B, Q, R)

In [6]:
'''
Basic implementation of a feed forward neural network with a single hidden layer. 

Takes input, a 1-d parameter vector, an activation function, and numbers of neurons at each layer. The parameter 
vector should be encoded as [W1.flatten(); B1; W2.flatten(); B2] where the Ws and Bs are the matrix weights and 
offset vectors. Activation function for the output layer is assumed to be linear.

'''

def simpleFeedForward(input, params, n_in, n_hidden, n_out, activation=np.tanh):
    
    ## Reshape our parameters to be used to calculate the output
    
    w1 = params[0 : n_in*n_hidden].reshape(n_hidden, n_in)
    b1 = params[n_in*n_hidden : n_in*n_hidden + n_hidden]
    w2 = params[n_in*n_hidden + n_hidden : n_in*n_hidden + n_hidden + n_hidden * n_out].reshape(n_out, n_hidden)
    b2 = params[n_in*n_hidden + n_hidden + n_hidden * n_out:]
    
    lin_midstep = np.dot(w1, input) + b1
    if activation == None:
        midstep = lin_midstep
    else: 
        midstep = activation(lin_midstep)
    
    output = np.dot(w2, midstep) + b2
    
    return output

In [11]:
'''
Function to evaluate the total penalty associated with a parameter vector theta for our simple feed forward network.

Simulates cartpole with the controller given by theta and computes the sum of costs, which are assumed to be a 
quadratic form of the distance from the current position x to the target position.

'''

def evaluate_theta(start, target, T, Q, params, n_in, n_hidden, n_out):

    dt = 0.1
    
    u = np.zeros(T)
    x = np.zeros([n_in, T+1])
    x[:, 0] = start
    for t in range(T):
        u[t] = simpleFeedForward(x[:, t], params, n_in, n_hidden, n_out)
        x[:, t+1] = sim_cartpole(x[:,t], u[t], dt)
    
    x_diff = x - np.transpose(np.tile(target, (x.shape[1],1)))
    penalty =  np.sum(np.diag(np.dot(np.transpose(x_diff), np.dot(Q, x_diff))))
    
    return penalty, x, u
    

In [14]:
'''
Some code to try to run policy gradient. Does not work yet - blows up to NaNs.

'''

##
## Initialize our variables
##
rng = np.random.RandomState(1234)
input = np.array([0, np.pi - np.pi/10, 0, 0])
n_in = 4
n_hidden = 20
n_out = 1

W1 = np.asarray(
                rng.uniform(
                    low = -np.sqrt(6. / (n_in + n_hidden)), 
                    high = np.sqrt(6. / (n_in + n_hidden)),
                    size = (n_in, n_hidden)
                ))

W2 = np.asarray(
                rng.uniform(
                    low = -np.sqrt(6. / (n_out + n_hidden)), 
                    high = np.sqrt(6. / (n_out + n_hidden)),
                    size = (n_hidden, n_out)
                ))

B1 = np.zeros([n_hidden,1])
B2 = np.zeros([n_out, 1])

## Cost function penalizes equal deviation from target coordinates
target = np.array([0, np.pi, 0, 0])
Q = np.eye(n_in)

def penalty(x):
    if len(x.shape) == 1:
        return np.dot(x-target,np.dot(Q, x-target))
    else:
        x_diff = x - np.transpose(np.tile(target, (x.shape[1],1)))
        return np.sum(np.diag(np.dot(np.transpose(x_diff), np.dot(Q, x_diff))))
    


params = np.append(np.append(np.append(W1.flatten(), B1), W2.flatten()), B2)

max_iter = 100

learning_rate = 0.0001

epsilon = 0.1

penalties = np.zeros(max_iter)

for i in range(max_iter):
    
    ## Generate trajectory to evaluate our policy

    start = np.array([0, np.pi - np.pi/10, 0, 0])
    
    penalties[i], x, u = evaluate_theta(start, target, 500, Q, params, n_in, n_hidden, n_out)
        
    ## Perform stochastic gradient descent
    
    # Choose random parameter to update
    direction = np.random.randint(0, len(params))
    
    unitv = np.zeros(len(params))
    unitv[direction] = epsilon
    
    # Calculate the penalty in that direction
    new_penalty, x, u = evaluate_theta(start, target, 500, Q, params + unitv, n_in, n_hidden, n_out)
    
    partial_x = unitv
    partial_x[direction] = (new_penalty - penalties[i])/epsilon
    
    ## Update the parameter vector
    params = params - learning_rate * partial_x

        

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: RuntimeWarning: overflow encountered in power
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:14: RuntimeWarning: overflow encountered in power
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:10: RuntimeWarning: invalid value encountered in sin
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:11: RuntimeWarning: invalid value encountered in cos


In [15]:
x[:,-1]

array([ nan,  nan,  nan,  nan])